# **SQL Database Coleta Log Contadores**

# 

- Carlos Eduardo Gimenes
- Last Modified: Junho, 2023

> **Copyright (C) 2023 Carlos Eduardo Gimenes**  
> All rights reserved.  
> You may alter this code for your own _non-commercial_ purposes.  
> You may republish altered code as long as you include this copyright and give due credit.
> 
> THIS CODE AND INFORMATION ARE PROVIDED "AS IS" WITHOUT WARRANTY OF ANY KIND, EITHER EXPRESSED OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE IMPLIED WARRANTIES OF MERCHANTABILITY AND/OR FITNESS FOR A PARTICULAR PURPOSE.

# **Legenda**

- **BatchRequests - ID 1**: Transações por segundo no SQL Server
- **User Connection** **\- ID 2**: Quantidade de conexões no Banco de Dados
- **CPU** **\- ID 3**: Consumo de CPU do Servidor
- **Page Life Expectancy** **\- ID 4**: Expectativa de vida em segundos de uma página na memória do SQL Server

# Create Database Traces

In [ ]:
/*
-------------------------------------------------------------------------------------------------------------------------
	Title		:	Create Database Traces
-------------------------------------------------------------------------------------------------------------------------
	Author		:	Gimenes
	Date		:	05/06/2023
	Requester	:	Gimenes
	Purpose		:	Database Coleta Log Contadores
	Program		:	Not applicable
    Credito     :   http://www.fabriciolima.net/
			
-------------------------------------------------------------------------------------------------------------------------
*/
-- >>> Inicio da Query
IF NOT EXISTS (
        SELECT name
        FROM sys.databases
        WHERE name = 'Traces'
        )
    CREATE DATABASE [Traces] CONTAINMENT = NONE ON PRIMARY (
        NAME = N'Traces_Data'
        , FILENAME = N'/mssql/data/Traces_Data.mdf'
        , SIZE = 1024000 KB
        , MAXSIZE = 1536000 KB
        , FILEGROWTH = 30 %
        ) LOG ON (
        NAME = N'Traces_log'
        , FILENAME = N'/mssql/data/Traces_log.ldf'
        , SIZE = 256000 KB
        , MAXSIZE = 512000 KB
        , FILEGROWTH = 30 %
        )
GO

ALTER DATABASE [Traces]

SET COMPATIBILITY_LEVEL = 150
GO

ALTER DATABASE [Traces]

SET ANSI_NULL_DEFAULT OFF
GO

ALTER DATABASE [Traces]

SET ANSI_NULLS OFF
GO

ALTER DATABASE [Traces]

SET ANSI_PADDING OFF
GO

ALTER DATABASE [Traces]

SET ANSI_WARNINGS OFF
GO

ALTER DATABASE [Traces]

SET ARITHABORT OFF
GO

ALTER DATABASE [Traces]

SET AUTO_CLOSE OFF
GO

ALTER DATABASE [Traces]

SET AUTO_SHRINK OFF
GO

ALTER DATABASE [Traces]

SET AUTO_CREATE_STATISTICS ON (INCREMENTAL = ON)
GO

ALTER DATABASE [Traces]

SET AUTO_UPDATE_STATISTICS ON
GO

ALTER DATABASE [Traces]

SET CURSOR_CLOSE_ON_COMMIT OFF
GO

ALTER DATABASE [Traces]

SET CURSOR_DEFAULT GLOBAL
GO

ALTER DATABASE [Traces]

SET CONCAT_NULL_YIELDS_NULL OFF
GO

ALTER DATABASE [Traces]

SET NUMERIC_ROUNDABORT OFF
GO

ALTER DATABASE [Traces]

SET QUOTED_IDENTIFIER OFF
GO

ALTER DATABASE [Traces]

SET RECURSIVE_TRIGGERS OFF
GO

ALTER DATABASE [Traces]

SET DISABLE_BROKER
GO

ALTER DATABASE [Traces]

SET AUTO_UPDATE_STATISTICS_ASYNC ON
GO

ALTER DATABASE [Traces]

SET DATE_CORRELATION_OPTIMIZATION OFF
GO

ALTER DATABASE [Traces]

SET PARAMETERIZATION SIMPLE
GO

ALTER DATABASE [Traces]

SET READ_COMMITTED_SNAPSHOT OFF
GO

ALTER DATABASE [Traces]

SET READ_WRITE
GO

ALTER DATABASE [Traces]

SET RECOVERY FULL
GO

ALTER DATABASE [Traces]

SET MULTI_USER
GO

ALTER DATABASE [Traces]

SET PAGE_VERIFY CHECKSUM
GO

ALTER DATABASE [Traces]

SET TARGET_RECOVERY_TIME = 60 SECONDS
GO

ALTER DATABASE [Traces]

SET DELAYED_DURABILITY = DISABLED
GO

USE [Traces]
GO

ALTER DATABASE SCOPED CONFIGURATION

SET LEGACY_CARDINALITY_ESTIMATION = OFF;
GO

ALTER DATABASE SCOPED CONFIGURATION
FOR SECONDARY

SET LEGACY_CARDINALITY_ESTIMATION = PRIMARY;
GO

ALTER DATABASE SCOPED CONFIGURATION

SET MAXDOP = 0;
GO

ALTER DATABASE SCOPED CONFIGURATION
FOR SECONDARY

SET MAXDOP = PRIMARY;
GO

ALTER DATABASE SCOPED CONFIGURATION

SET PARAMETER_SNIFFING = ON;
GO

ALTER DATABASE SCOPED CONFIGURATION
FOR SECONDARY

SET PARAMETER_SNIFFING = PRIMARY;
GO

ALTER DATABASE SCOPED CONFIGURATION

SET QUERY_OPTIMIZER_HOTFIXES = OFF;
GO

ALTER DATABASE SCOPED CONFIGURATION
FOR SECONDARY

SET QUERY_OPTIMIZER_HOTFIXES = PRIMARY;
GO

USE [Traces]
GO

IF NOT EXISTS (
        SELECT name
        FROM sys.filegroups
        WHERE is_default = 1
            AND name = N'PRIMARY'
        )
    ALTER DATABASE [Traces] MODIFY FILEGROUP [PRIMARY] DEFAULT
GO

-- <<< Fim da Query


# Create table to store Log Contadores

In [4]:
/*
-------------------------------------------------------------------------------------------------------------------------
	Title		:	Create table to store Log Contadores
-------------------------------------------------------------------------------------------------------------------------
	Author		:	Gimenes
	Date		:	22/06/2023
	Requester	:	Gimenes
	Purpose		:	Database Coleta Log Contadores
	Program		:	Not applicable
    Credito     :   http://www.fabriciolima.net/
			
-------------------------------------------------------------------------------------------------------------------------
*/
-- >>> Inicio da Query

USE Traces
GO

IF OBJECT_ID('Contador') IS NOT NULL
    DROP TABLE Contador

IF OBJECT_ID('Registro_Contador') IS NOT NULL
    DROP TABLE Registro_Contador

CREATE TABLE Contador (
    Id_Contador INT identity
    , Nm_Contador VARCHAR(50)
    )

INSERT INTO Contador (Nm_Contador)
SELECT 'BatchRequests'

INSERT INTO Contador (Nm_Contador)
SELECT 'User_Connection'

INSERT INTO Contador (Nm_Contador)
SELECT 'CPU'

INSERT INTO Contador (Nm_Contador)
SELECT 'Page Life Expectancy'

SELECT *
FROM Contador

CREATE TABLE [dbo].[Registro_Contador] (
    [Id_Registro_Contador] [int] IDENTITY(1, 1) NOT NULL
    , [Dt_Log] [datetime] NULL
    , [Id_Contador] [int] NULL
    , [Valor] [int] NULL
    ) ON [PRIMARY]

-- <<< Fim da Query


Commands completed successfully.

(1 row affected)

(1 row affected)

(1 row affected)

(1 row affected)

(4 rows affected)

Total execution time: 00:00:00.342

Id_Contador,Nm_Contador
1,BatchRequests
2,User_Connection
3,CPU
4,Page Life Expectancy


# Create Procedure to capture data

Make sure you are connected a Traces database

In [5]:
/*
-------------------------------------------------------------------------------------------------------------------------
	Title		:	Create Procedure to capture data
-------------------------------------------------------------------------------------------------------------------------
	Author		:	Gimenes
	Date		:	22/06/2023
	Requester	:	Gimenes
	Purpose		:	Database Coleta Log Contadores
	Program		:	Not applicable
    Credito     :   http://www.fabriciolima.net/
			
-------------------------------------------------------------------------------------------------------------------------
*/
-- >>> Inicio da Query

USE Traces
GO

IF OBJECT_ID('stpCarga_ContadoresSQL') IS NOT NULL
    DROP PROCEDURE stpCarga_ContadoresSQL
GO

CREATE PROCEDURE stpCarga_ContadoresSQL
AS
BEGIN
    DECLARE @BatchRequests INT
        , @User_Connection INT
        , @CPU INT
        , @PLE INT
    DECLARE @RequestsPerSecondSample1 BIGINT
    DECLARE @RequestsPerSecondSample2 BIGINT

    SELECT @RequestsPerSecondSample1 = cntr_value
    FROM sys.dm_os_performance_counters
    WHERE counter_name = 'Batch Requests/sec'

    WAITFOR DELAY '00:00:05'

    SELECT @RequestsPerSecondSample2 = cntr_value
    FROM sys.dm_os_performance_counters
    WHERE counter_name = 'Batch Requests/sec'

    SELECT @BatchRequests = (@RequestsPerSecondSample2 - @RequestsPerSecondSample1) / 5

    SELECT @User_Connection = cntr_Value
    FROM sys.dm_os_performance_counters
    WHERE counter_name = 'User Connections'

    SELECT TOP (1) @CPU = (SQLProcessUtilization + (100 - SystemIdle - SQLProcessUtilization))
    FROM (
        SELECT record.value('(./Record/@id)[1]', 'int') AS record_id
            , record.value('(./Record/SchedulerMonitorEvent/SystemHealth/SystemIdle)[1]', 'int') AS [SystemIdle]
            , record.value('(./Record/SchedulerMonitorEvent/SystemHealth/ProcessUtilization)[1]', 'int') AS [SQLProcessUtilization]
            , [timestamp]
        FROM (
            SELECT [timestamp]
                , CONVERT(XML, record) AS [record]
            FROM sys.dm_os_ring_buffers
            WHERE ring_buffer_type = N'RING_BUFFER_SCHEDULER_MONITOR'
                AND record LIKE '%<SystemHealth>%'
            ) AS x
        ) AS y

    SELECT @PLE = cntr_value
    FROM sys.dm_os_performance_counters
    WHERE counter_name = 'Page life expectancy'
        AND object_name LIKE '%Buffer Manager%'

    INSERT INTO Registro_Contador (
        Dt_Log
        , Id_Contador
        , Valor
        )
    SELECT GETDATE()
        , 1
        , @BatchRequests

    INSERT INTO Registro_Contador (
        Dt_Log
        , Id_Contador
        , Valor
        )
    SELECT GETDATE()
        , 2
        , @User_Connection

    INSERT INTO Registro_Contador (
        Dt_Log
        , Id_Contador
        , Valor
        )
    SELECT GETDATE()
        , 3
        , @CPU

    INSERT INTO Registro_Contador (
        Dt_Log
        , Id_Contador
        , Valor
        )
    SELECT GETDATE()
        , 4
        , @PLE
END

-- <<< Fim da Query

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.070

# Create Job to process Coleta Log Contadores

# 

Make sure you are connected a msdb system database

In [6]:
/*
-------------------------------------------------------------------------------------------------------------------------
	Title		:	Creation of Job to process Coleta Log Contadores
-------------------------------------------------------------------------------------------------------------------------
	Author		:	Gimenes
	Date		:	22/06/2023
	Requester	:	Gimenes
	Purpose		:	Database Coleta Log Contadores
	Program		:	Not applicable
    Credito     :   Nenhum
			
-------------------------------------------------------------------------------------------------------------------------
*/
-- >>> Inicio da Query

USE [msdb]
GO

/****** Object:  Job [DBA - Coleta Log Contadores]    Script Date: 22/06/2023 11:37:02 ******/
BEGIN TRANSACTION
DECLARE @ReturnCode INT
SELECT @ReturnCode = 0
/****** Object:  JobCategory [Database Maintenance]    Script Date: 22/06/2023 11:37:03 ******/
IF NOT EXISTS (SELECT name FROM msdb.dbo.syscategories WHERE name=N'Database Maintenance' AND category_class=1)
BEGIN
EXEC @ReturnCode = msdb.dbo.sp_add_category @class=N'JOB', @type=N'LOCAL', @name=N'Database Maintenance'
IF (@@ERROR <> 0 OR @ReturnCode <> 0) GOTO QuitWithRollback

END

DECLARE @jobId BINARY(16)
EXEC @ReturnCode =  msdb.dbo.sp_add_job @job_name=N'DBA - Coleta Log Contadores', 
		@enabled=1, 
		@notify_level_eventlog=0, 
		@notify_level_email=0, 
		@notify_level_netsend=0, 
		@notify_level_page=0, 
		@delete_level=0, 
		@category_name=N'Database Maintenance', 
		@owner_login_name=N'carlos.gimenes', @job_id = @jobId OUTPUT
IF (@@ERROR <> 0 OR @ReturnCode <> 0) GOTO QuitWithRollback
/****** Object:  Step [Executa SP]    Script Date: 22/06/2023 11:37:03 ******/
EXEC @ReturnCode = msdb.dbo.sp_add_jobstep @job_id=@jobId, @step_name=N'Executa SP', 
		@step_id=1, 
		@cmdexec_success_code=0, 
		@on_success_action=1, 
		@on_success_step_id=0, 
		@on_fail_action=2, 
		@on_fail_step_id=0, 
		@retry_attempts=0, 
		@retry_interval=0, 
		@os_run_priority=0, @subsystem=N'TSQL', 
		@command=N'EXEC dbo.stpCarga_ContadoresSQL', 
		@database_name=N'Traces', 
		@flags=8
IF (@@ERROR <> 0 OR @ReturnCode <> 0) GOTO QuitWithRollback
EXEC @ReturnCode = msdb.dbo.sp_update_job @job_id = @jobId, @start_step_id = 1
IF (@@ERROR <> 0 OR @ReturnCode <> 0) GOTO QuitWithRollback
EXEC @ReturnCode = msdb.dbo.sp_add_jobschedule @job_id=@jobId, @name=N'A cada 1 minuto', 
		@enabled=1, 
		@freq_type=4, 
		@freq_interval=1, 
		@freq_subday_type=4, 
		@freq_subday_interval=1, 
		@freq_relative_interval=0, 
		@freq_recurrence_factor=0, 
		@active_start_date=20190218, 
		@active_end_date=99991231, 
		@active_start_time=112, 
		@active_end_time=235959, 
		@schedule_uid=N'41234bdc-f187-40e7-8998-212cc6a636e6'
IF (@@ERROR <> 0 OR @ReturnCode <> 0) GOTO QuitWithRollback
EXEC @ReturnCode = msdb.dbo.sp_add_jobserver @job_id = @jobId, @server_name = N'(local)'
IF (@@ERROR <> 0 OR @ReturnCode <> 0) GOTO QuitWithRollback
COMMIT TRANSACTION
GOTO EndSave
QuitWithRollback:
    IF (@@TRANCOUNT > 0) ROLLBACK TRANSACTION
EndSave:
GO




-- <<< Fim da Query


Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.626

* * *

# **Fim de arquivo**